<a href="https://colab.research.google.com/github/as9786/ComputerVision/blob/main/ImageGeneration/code/PixelCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm

# Dataset

In [2]:
num_classes = 10
input_shape = (28,28,1)
n_residual_blocks = 5
(x,_),(y,_) = keras.datasets.mnist.load_data()
data = np.concatenate((x,y),axis=0)
# 0과 1의 값을 갖도록 정규화화
data = np.where(data < (0.33 * 256),0,1)
data = data.astype(np.float32)

11490434/11490434 [==============================] - 2s 0us/step


# Model

In [3]:
class PixelConvLayer(layers.Layer):
  def __init__(self,mask_type,**kwargs):
    super().__init__()
    self.mask_type = mask_type
    self.conv = layers.Conv2D(**kwargs)

  def build(self,input_shape):
    self.conv.build(input_shape)
    kernel_shape = self.conv.kernel.get_shape()
    self.mask = np.zeros(shape=kernel_shape)
    self.mask[:kernel_shape[0]//2,...] = 1.0
    self.mask[kernel_shape[0]//2,:kernel_shape[1]//2,...] = 1.0
    if self.mask_type=='B':
      self.mask[kernel_shape[0]//2,kernel_shape[1]//2,...] = 1.0
  
  def call(self,inputs):
    self.conv.kernel.assign(self.conv.kernel*self.mask)
    return self.conv(inputs)

In [10]:
example_kernel = np.zeros((3,3))
example_kernel

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [11]:
# Mask A
example_kernel[:3//2,...] = 1.
example_kernel[3//2,:3//2,...]=1.
example_kernel

array([[1., 1., 1.],
       [1., 0., 0.],
       [0., 0., 0.]])

In [13]:
# Mask B
example_kernel[3//2,3//2,...] = 1
example_kernel

array([[1., 1., 1.],
       [1., 1., 0.],
       [0., 0., 0.]])

In [4]:
class ResidualBlock(layers.Layer):
  def __init__(self,filters,**kwargs):
    super().__init__(**kwargs)
    self.conv1 = layers.Conv2D(filters=filters,kernel_size=1,activation='relu')
    self.pixel_conv = PixelConvLayer(
        mask_type = 'B',
        filters=filters//2,
        kernel_size=3,
        activation='relu',
        padding='same'
    )
    self.conv2 = keras.layers.Conv2D(filters=filters,kernel_size=1,activation='relu')

  def call(self,inputs):
    x = self.conv1(inputs)
    x = self.pixel_conv(x)
    x = self.conv2(x)
    return layers.add([inputs,x])

In [5]:
inputs = keras.Input(shape=input_shape)
x = PixelConvLayer(mask_type='A',filters=128,kernel_size=7,activation='relu',padding='same')(inputs)

for _ in range(n_residual_blocks):
  x = ResidualBlock(filters=128)(x)

for _ in range(2):
  x = PixelConvLayer(
      mask_type='B',
      filters = 128,
      kernel_size=1,
      strides=1,
      activation='relu',
      padding='valid'
  )(x)

out = layers.Conv2D(filters=1,kernel_size=1,strides=1,activation='sigmoid',padding='valid')(x)

In [6]:
pixel_cnn = keras.Model(inputs,out)
adam = keras.optimizers.Adam(learning_rate = 0.0005)
pixel_cnn.compile(optimizer=adam,loss='binary_crossentropy')
pixel_cnn.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 pixel_conv_layer (PixelConv  (None, 28, 28, 128)      6400      
 Layer)                                                          
                                                                 
 residual_block (ResidualBlo  (None, 28, 28, 128)      98624     
 ck)                                                             
                                                                 
 residual_block_1 (ResidualB  (None, 28, 28, 128)      98624     
 lock)                                                           
                                                                 
 residual_block_2 (ResidualB  (None, 28, 28, 128)      98624     
 lock)                                                       

In [14]:
pixel_cnn.fit(x=data,y=data,batch_size=128,epochs=50,validation_split=0.1,verbose=2)

Epoch 1/50
493/493 - 53s - loss: 0.1132 - val_loss: 0.0927 - 53s/epoch - 107ms/step
Epoch 2/50
493/493 - 51s - loss: 0.0912 - val_loss: 0.0898 - 51s/epoch - 104ms/step
Epoch 3/50
493/493 - 52s - loss: 0.0891 - val_loss: 0.0885 - 52s/epoch - 105ms/step
Epoch 4/50
493/493 - 51s - loss: 0.0880 - val_loss: 0.0880 - 51s/epoch - 104ms/step
Epoch 5/50
493/493 - 51s - loss: 0.0871 - val_loss: 0.0868 - 51s/epoch - 104ms/step
Epoch 6/50
493/493 - 52s - loss: 0.0866 - val_loss: 0.0872 - 52s/epoch - 105ms/step
Epoch 7/50
493/493 - 51s - loss: 0.0862 - val_loss: 0.0860 - 51s/epoch - 104ms/step
Epoch 8/50
493/493 - 51s - loss: 0.0858 - val_loss: 0.0859 - 51s/epoch - 104ms/step
Epoch 9/50
493/493 - 51s - loss: 0.0854 - val_loss: 0.0855 - 51s/epoch - 104ms/step
Epoch 10/50
493/493 - 51s - loss: 0.0852 - val_loss: 0.0852 - 51s/epoch - 104ms/step
Epoch 11/50
493/493 - 52s - loss: 0.0849 - val_loss: 0.0849 - 52s/epoch - 105ms/step
Epoch 12/50
493/493 - 51s - loss: 0.0847 - val_loss: 0.0849 - 51s/epoch - 

# Infer

In [15]:
from IPython.display import Image, display

# Create an empty array of pixels.
batch = 4
pixels = np.zeros(shape=(batch,) + (pixel_cnn.input_shape)[1:])
batch, rows, cols, channels = pixels.shape

# Iterate over the pixels because generation has to be done sequentially pixel by pixel.
for row in tqdm(range(rows)):
    for col in range(cols):
        for channel in range(channels):
            # Feed the whole array and retrieving the pixel value probabilities for the next
            # pixel.
            probs = pixel_cnn.predict(pixels)[:, row, col, channel]
            # Use the probabilities to pick pixel values and append the values to the image
            # frame.
            pixels[:, row, col, channel] = tf.math.ceil(
                probs - tf.random.uniform(probs.shape)
            )

def deprocess_image(x):
    # Stack the single channeled black and white image to RGB values.
    x = np.stack((x, x, x), 2)
    # Undo preprocessing
    x *= 255.0
    # Convert to uint8 and clip to the valid range [0, 255]
    x = np.clip(x, 0, 255).astype("uint8")
    return x

# Iterate over the generated images and plot them with matplotlib.
for i, pic in enumerate(pixels):
    keras.preprocessing.image.save_img(
        "generated_image_{}.png".format(i), deprocess_image(np.squeeze(pic, -1))
    )

display(Image("generated_image_0.png"))
display(Image("generated_image_1.png"))
display(Image("generated_image_2.png"))
display(Image("generated_image_3.png"))

  0%|          | 0/28 [00:00<?, ?it/s]

1/1 [==============================] - 0s 28ms/step


  4%|▎         | 1/28 [00:03<01:35,  3.54s/it]

1/1 [==============================] - 0s 114ms/step


  7%|▋         | 2/28 [00:06<01:17,  2.99s/it]

...

 93%|█████████▎| 26/28 [00:47<00:03,  1.81s/it]

1/1 [==============================] - 0s 19ms/step


 96%|█████████▋| 27/28 [00:49<00:01,  1.75s/it]

1/1 [==============================] - 0s 20ms/step


100%|██████████| 28/28 [00:50<00:00,  1.82s/it]
